In [1]:
import pandas as pd
import numpy as np
import re
import string
import tensorflow as tf
from tqdm import tqdm
 
import nltk
#en_stop = set(nltk.corpus.<a href="https://thinkinfi.com/how-to-remove-stop-words-in-python/" data-internallinksmanager029f6b8e52c="5" title="How To Remove Stop words In Python" target="_blank" rel="noopener">stopwords</a>.words('english'))
 
from gensim.models import FastText
 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
 
# <a href="https://thinkinfi.com/difference-between-stemming-and-lemmatizing-and-where-to-use/" data-internallinksmanager029f6b8e52c="7" title="Difference between stemming and lemmatizing and where to use" target="_blank" rel="noopener">Lemmatization</a>
from nltk import WordNetLemmatizer
stemmer = WordNetLemmatizer()

In [2]:
# import tensorflow as tf
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before GPUs have been initialized
#     print(e)

In [3]:
#train_df = pd.read_csv("Mal_sentiment_full_train.tsv", header=None, sep='\t')
train_df = pd.read_csv("Mal_sentiment_full_train.tsv", sep='\t')
print(len(train_df))

#test_df = pd.read_csv("Mal_sentiment_full_dev.tsv", header=None, sep='\t')
valid_df = pd.read_csv("Mal_sentiment_full_dev.tsv", sep='\t')
print(len(valid_df))

test_df = pd.read_csv("Mal_sentiment_full_test_withoutlabels.tsv", sep='\t')
len(test_df)

15888
1766


1962

In [4]:
test_df

,id,text
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?
3,Mal_4,Nowadays 944k views is considered as 1M views ...
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......
...,...,...
1957,Mal_1958,Expression വച്ചു നോക്കുമ്പോൾ മൂക്കുത്തി കട്ടത്...
1958,Mal_1959,എനിക്ക് ഒരു 100 like തരുമോ
1959,Mal_1960,Nannayi onnu poliyunna lakshnm unde...Thrill m...
1960,Mal_1961,Athikam eduthu kayatti vekkenda....direction k...


In [5]:
#removing all the emoji from the given data

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)


for i in range(len(train_df)):
    temp1 = deEmojify(train_df['text'][i])
    train_df['text'][i] = temp1

for j in range(len(valid_df)):
    temp2 = deEmojify(valid_df['text'][j])
    valid_df['text'][j] = temp2

for k in range(len(test_df)):
    temp3 = deEmojify(test_df['text'][k])
    test_df['text'][k]=temp3

In [6]:
def preprocess_data(text):
  user_name = '@[A-Za-z0-9]+' # Pattern for matching the user names in the tweet
  has_tag = '#[A-Za-z0-9]+' # Pattern for matching the hasgtags in the tweer
  url = 'https?:\/\/[A-Za-z0-9\.\/\-]+' # Pattern for matching the URLs in the tweet
  #emoji_pattern = r'/[x{1F600}-x{1F64F}]/u'
  
  x = text.lower()  # Lower casing all the characters
  x = re.sub(user_name, '', x) # Replace the username with an empty characrer
  x = re.sub(has_tag,'',x) # Replace the hashtags with an empty characrer
  x = re.sub(url, '', x) # Replace the URLs with an empty characrer
  #x = re.sub(emoji_pattern, '', x)
  x = x.translate(str.maketrans('', '', string.punctuation)) # Replace all the characters except alphabets and digits from the tweet

  return x

In [7]:
train_df['text'] = train_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe
valid_df['text'] = valid_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe
test_df['text'] = test_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe

In [8]:
train_text = train_df['text'].tolist()
valid_text = valid_df['text'].tolist()
test_text = test_df['text'].tolist()

train_label = train_df['category'].tolist()
valid_label = valid_df['category'].tolist()

In [9]:
import collections
from collections import defaultdict, Counter

In [10]:
class_dist2 = Counter(valid_label)
class_dist2

Counter({'Positive': 706,
         'not-malayalam': 141,
         'unknown_state': 580,
         'Negative': 237,
         'Mixed_feelings': 102})

In [11]:
class_dist = Counter(train_label)
class_dist

Counter({'unknown_state': 5279,
         'Positive': 6421,
         'Negative': 2105,
         'Mixed_feelings': 926,
         'not-malayalam': 1157})

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
le = LabelEncoder()
train_label = le.fit_transform(train_label)
train_label

array([4, 2, 4, ..., 4, 2, 0], dtype=int64)

In [14]:
valid_label = le.fit_transform(valid_label)
valid_label

array([2, 3, 2, ..., 4, 2, 2], dtype=int64)

In [15]:
le.inverse_transform([0,1,2,3,4])

array(['Mixed_feelings', 'Negative', 'Positive', 'not-malayalam',
       'unknown_state'], dtype='<U14')

In [16]:
train_df.isnull().sum()

text        0
category    0
dtype: int64

In [17]:
train_text

['sunny chechi fans evide like adichu power kaniku',
 'angane july month ile ende aadyathe leave njan angu edukuva july 5 nu',
 'ഏട്ടന്റെ പുതിയ പടത്തിനു വേണ്ടി കാത്തിരിക്കുന്നവർ ലൈക്\u200c ബട്ടൺ അടിച്ചു പോളിക്കോ',
 'ഇനി ലാലേട്ടന്റെ വേട്ട തുടങ്ങാൻ പോകുന്നു മുൻ കാല collection recordukal ഭേതിച്ചിരിക്കും',
 'trailer powli oru raksha illa  pakshea padam irangattea veruthea over degrade venda padam irangittu paraya',
 'malayalam padam thanne aanu 2020 vere level',
 'history of mammukka   kattawaiting and support love u mammukka',
 'yaaru andha mech le penne ',
 'ആമയെ വെള്ളത്തിൽ മുക്കി കൊല്ലുന്ന ഇനമാ ',
 'ജോഷി രണ്ടുംകല്പിച്ചു ആണെല്ലോ സംഭവം എന്തായാലും പൊളിക്കും',
 'ഇതിൽ സ്കോർ ചെയ്യാൻ പോവുന്നത് സുഷിൻ ശ്യാം തന്നെ',
 'സ്വാസിക സ്വാസു love u dear all the best മുത്തേ all kerala swasika fans association',
 'ഒരു കാതിൽ ഓല ഞാൻ കണ്ടീല  ചെക്ക്',
 'siddique nde hit aaya last movie ethaarunnu',
 'ആദ്യ നൂറു കോടി വേണ്ടവർ adi mwonoose like',
 'ഈ സിനിമ കാണണം എന്ന് ആഗ്രഹം ഉള്ള എത്ര പേരുണ്ട് ഇവിടെ',
 'ithinidakk 

In [18]:
# from transformers import AutoModel
# model2 = AutoModel.from_pretrained('neuralspace-reverie/indic-transformers-hi-bert')

In [19]:
from transformers import TFAutoModel
model = TFAutoModel.from_pretrained('neuralspace-reverie/indic-transformers-hi-bert')

Some layers from the model checkpoint at neuralspace-reverie/indic-transformers-hi-bert were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at neuralspace-reverie/indic-transformers-hi-bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [20]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('neuralspace-reverie/indic-transformers-hi-bert')

In [21]:
input_ids = tokenizer(train_text, return_tensors='tf',truncation=True,padding=True)['input_ids']

#input_ids = tokenizer(valid_text, return_tensors='tf',truncation=True,padding=True)['input_ids']

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [22]:
import math

In [23]:
Batch_size = 64
start = 0;Endu = Batch_size
fullsize = math.floor(len(input_ids)/Batch_size)

In [24]:
import time

In [25]:
for i in range(fullsize):
    if i+1 == fullsize:
        Endu = len(input_ids)
        out = model(input_ids[int(start):int(Endu)])
    else:
        out = model(input_ids[int(start):int(Endu)])
    if i == 0:
        total = out.pooler_output 
    else:
        #torch.cat((total,out.last_hidden_state),0)
        total = tf.concat([total, out.pooler_output],0)
    start = Batch_size * (i+1)
    Endu = Batch_size * (i+2)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247


In [26]:
len(total)

15888

In [27]:
total

<tf.Tensor: shape=(15888, 768), dtype=float32, numpy=
array([[ 0.13898624,  0.8632123 , -0.41412416, ..., -0.24006976,
        -0.2516841 , -0.18612398],
       [ 0.21298957,  0.8876664 , -0.33245718, ..., -0.23593758,
        -0.17669284, -0.13031483],
       [ 0.17121328,  0.8713102 , -0.3918358 , ..., -0.24652243,
        -0.22326042, -0.1780453 ],
       ...,
       [ 0.05744632,  0.8334568 , -0.45792982, ..., -0.23478264,
        -0.3150851 , -0.21995145],
       [ 0.29268438,  0.90130556, -0.27445528, ..., -0.2461739 ,
        -0.10207261, -0.10287685],
       [ 0.175603  ,  0.86988324, -0.3683278 , ..., -0.20924659,
        -0.28820264, -0.17048363]], dtype=float32)>

In [28]:
list(out)

['last_hidden_state', 'pooler_output']

In [29]:
#out['last_hidden_state']

In [30]:
# input_ids.

In [31]:
!nvidia-smi

Fri Oct 22 03:55:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.13       Driver Version: 496.13       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   71C    P0    41W /  N/A |   5330MiB /  6144MiB |     79%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [32]:
X_train = np.array(total)
np.save('Indice_BERT_X_train.npy', X_train)

In [33]:
# X_valid = np.array(total)
# np.save('Indice_BERT_X_val.npy', X_valid)

In [34]:
# Y_train = np.array(train_label)
# np.save('Indice_BERT_Y_train.npy',Y_train)

# Y_val = np.array(valid_label)
# np.save('Indice_BERT_Y_val.npy',Y_val)

In [42]:
ls

 Volume in drive C is Acer
 Volume Serial Number is 3818-7542

 Directory of C:\Users\phvpa\OneDrive - Amrita Vishwa Vidyapeetham\Amrita-Mtech-CEN\ProjectWork\MTech Thesis\Denoise in Code mix data\Experiments\Malyalam

22-10-2021  03:57    <DIR>          .
22-10-2021  03:50    <DIR>          ..
20-10-2021  11:55    <DIR>          .ipynb_checkpoints
21-10-2021  01:01    <DIR>          Colab_work
21-10-2021  08:10            13,606 FastText.ipynb
22-10-2021  03:56    <DIR>          Indice_BERT
22-10-2021  03:57           186,790 Indice_BERT.ipynb
22-10-2021  03:55        48,808,064 Indice_BERT_X_train.npy
22-10-2021  03:50         5,425,280 Indice_BERT_X_val.npy
22-10-2021  03:50           127,232 Indice_BERT_Y_train.npy
22-10-2021  03:50            14,256 Indice_BERT_Y_val.npy
23-05-2021  09:42           268,731 Mal_sentiment_full_dev.tsv
16-06-2021  04:34           287,576 Mal_sentiment_full_test_withoutlabels.tsv
23-05-2021  09:41         2,382,197 Mal_sentiment_full_train.tsv
09-10-2